In [18]:
# Import required libraries for file handling and data processing
import os
import json
import csv
import zipfile


In [2]:
# Path to the provided dataset (mainfolder.zip)
zip_path = r"C:\Users\shali\OneDrive\Desktop\Python Upgrad\mainfolder.zip"

# Path where files will be extracted
extract_path = r"C:\Users\shali\OneDrive\Desktop\Python Upgrad"

# Unzip the dataset into the extract path
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully!")


Unzipped successfully!


In [10]:
# Define the main folder and its subfolders
main_folder = r"C:\Users\shali\OneDrive\Desktop\Python Upgrad\mainfolder"

sales_file = os.path.join(main_folder, "sales_data.csv")
details_folder = os.path.join(main_folder, "product_details")
descriptions_folder = os.path.join(main_folder, "product_descriptions")

# Verify that the main folder contains the expected files and folders
print(os.listdir(main_folder))  # Check contents


['product_descriptions', 'product_details', 'sales_data.csv']


In [11]:
# Function to load data from CSV, JSON, and TXT files into dictionaries
def load_data(main_folder):
    sales_file = os.path.join(main_folder, "sales_data.csv")
    details_folder = os.path.join(main_folder, "product_details")
    descriptions_folder = os.path.join(main_folder, "product_descriptions")
    
    # Load sales data (CSV → dictionary with SKU as key)
    sales_data = {}
    with open(sales_file, mode="r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            sku = row["Product_SKU"]
            sales_data[sku] = [int(row[f"Day{i}"]) for i in range(1, 15)]
    
   # Load product details (JSON files → dictionary with SKU as key)
    product_details = {}
    for file in os.listdir(details_folder):
        if file.endswith(".json"):
            with open(os.path.join(details_folder, file), "r") as f:
                data = json.load(f)
                sku = file.replace("details_", "").replace(".json", "")
                product_details[sku] = data
    
    # Load product descriptions (TXT files → dictionary with SKU as key)
    product_descriptions = {}
    for file in os.listdir(descriptions_folder):
        if file.endswith(".txt"):
            with open(os.path.join(descriptions_folder, file), "r") as f:
                sku = file.replace("description_", "").replace(".txt", "")
                product_descriptions[sku] = f.read().strip()
    
    return sales_data, product_details, product_descriptions

# Load all data into dictionaries
sales_data, product_details, product_descriptions = load_data(main_folder)

# Display samples to confirm loading worked
print("Sales Data Sample:", list(sales_data.items())[:1])
print("Product Details Sample:", list(product_details.items())[:1])
print("Product Descriptions Sample:", list(product_descriptions.items())[:1])


Sales Data Sample: [('AISJDKFJW93NJ', [10, 12, 15, 18, 20, 22, 25, 28, 26, 30, 32, 29, 27, 24])]
Product Details Sample: [('AISJDKFJW93NJ', {'product_name': 'Wall Art Print', 'brand': 'ArtCraft', 'model': 'NatureCanvas-1001', 'specifications': 'Canvas print, Ready to hang', 'price': '$49.99', 'availability': 'In stock'})]
Product Descriptions Sample: [('AISJDKFJW93NJ', "Transform your living space with ArtCraft's NatureCanvas-1001 Wall Art Print.\nThis canvas print, ready to hang, brings the beauty of nature into your home.\nWith dimensions of 16 x 20 inches and a 4.6/5 stars rating, it's a stunning addition to your decor, creating a focal point that captures attention and sparks conversation.")]


In [12]:
# Function to update sales data for a given SKU
def update_sales_data(sales_data, sku, sales):
    sales_data[sku] = sales
    print(f"✅ Sales data updated for {sku}")
    return sales_data

# Function to update product details for a given SKU
def update_product_details(product_details, sku, details):
    product_details[sku] = details
    print(f"✅ Product details updated for {sku}")
    return product_details

# Function to update product description for a given SKU
def update_product_description(product_descriptions, sku, description):
    product_descriptions[sku] = description
    print(f"✅ Product description updated for {sku}")
    return product_descriptions


In [16]:
# Master function to update sales data, product details, and product description
# Ensures SKU length = 13, sales data = 14 numbers, and collects all product info
def update(sales_data, product_details, product_descriptions):
    # Validate SKU
    sku = input("Enter SKU (13 characters): ")
    if len(sku) != 13:
        print("❌ Error: SKU must be exactly 13 characters")
        return sales_data, product_details, product_descriptions
    
    # Validate sales data
    sales_input = input("Enter 14 sales values separated by spaces: ")
    sales = sales_input.split()
    if len(sales) != 14 or not all(s.isdigit() for s in sales):
        print("❌ Error: Must enter 14 whole numbers")
        return sales_data, product_details, product_descriptions
    sales = list(map(int, sales))
    
    # Collect product details
    name = input("Enter Product Name: ")
    brand = input("Enter Brand: ")
    model = input("Enter Model: ")
    specifications = input("Enter Specifications: ")
    price = input("Enter Price: ")
    availability = input("Enter Availability (In stock/Out of stock): ")
    details = {
        "Product Name": name,
        "Brand": brand,
        "Model": model,
        "Specifications": specifications,
        "Price": price,
        "Availability": availability
    }
    
    # Collect product description
    description = input("Enter Product Description: ")
    
    # Update all dictionaries
    update_sales_data(sales_data, sku, sales)
    update_product_details(product_details, sku, details)
    update_product_description(product_descriptions, sku, description)
    
    print("✅ Product successfully updated!")
    return sales_data, product_details, product_descriptions


In [17]:
# Function to save all updated dictionaries back into CSV, JSON, and TXT files
def dump_data(sales_data, product_details, product_descriptions, main_folder):
    sales_file = os.path.join(main_folder, "sales_data.csv")
    details_folder = os.path.join(main_folder, "product_details")
    descriptions_folder = os.path.join(main_folder, "product_descriptions")

    # Function to save all updated dictionaries back into CSV, JSON, and TXT files
    os.makedirs(details_folder, exist_ok=True)
    os.makedirs(descriptions_folder, exist_ok=True)
    
    # Save sales data into CSV
    with open(sales_file, mode="w", newline="") as f:
        fieldnames = ["Product_SKU"] + [f"Day{i}" for i in range(1, 15)]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for sku, sales in sales_data.items():
            row = {"Product_SKU": sku}
            for i, value in enumerate(sales, start=1):
                row[f"Day{i}"] = value
            writer.writerow(row)
    
    # Save each product detail into JSON file
    for sku, details in product_details.items():
        file_path = os.path.join(details_folder, f"details_{sku}.json")
        with open(file_path, "w") as f:
            json.dump(details, f, indent=4)
    
    # Save each product description into TXT file
    for sku, description in product_descriptions.items():
        file_path = os.path.join(descriptions_folder, f"description_{sku}.txt")
        with open(file_path, "w") as f:
            f.write(description)
    
    print("✅ All data dumped successfully!")


In [15]:
# Run this to interactively update product information
sales_data, product_details, product_descriptions = update(
    sales_data, product_details, product_descriptions
)

# Save the updated data back into files
dump_data(sales_data, product_details, product_descriptions, main_folder)


Enter SKU (13 characters):  SON9876543218
Enter 14 sales values separated by spaces:  33 44 55 66 77 88 99 1 2 3 5 6 7 8 
Enter Product Name:  samsung
Enter Brand:  galaxy
Enter Model:  s33
Enter Specifications:  128tb
Enter Price:  89999
Enter Availability:  In stock
Enter Product Description:  face beauty


✅ Sales data updated for SON9876543218
✅ Product details updated for SON9876543218
✅ Product description updated for SON9876543218
✅ Product successfully updated!
✅ All data dumped successfully!
